# Intro to Linear Programming

In [14]:
import numpy as np
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from scipy.optimize import linprog
from itertools import combinations
from IPython.display import display

# Problem 1

### Problem Statement

Consider a system of $m$ equations and $n$ unknowns:

$$Ax = b \hspace{25px} A \in \mathbb{R}^{mxn}, x \in \mathbb{R}^n, b \in \mathbb{R}^m$$

A basic solution of the system of equations is one in which at most there are $m$ variables different from zero.

a) Write a function that takes as input parameters the matrix $A$ and the vector $b$ of the aforementioned system of equations. The function should return a dictionary with 2 lists, the first containing a list of all basic solutions, and the second one containing a list of the bases corresponding to each basic solution. Use the following nomenclature for the function and the directory respectively:

```
directory = {'Solution':[], 'Base':[]}
```

Where each element of `directory['Solution']` is a vector in $\mathbb{R}^n$ with at least $n-m$ variables equal to 0 and each element of `directory['Base']` is an array in $\mathbb{R}^{mxn}$ with at least $n-m$ columns equal to 0 (columns not belonging to the used base).

b) Generate two matrices, $A, B \in \mathbb{R}^{3x5}$, one with full rank and one with deficient rank. Use the `pandas` library to convert a dictionary into a `DataFrame`. What are the differences in the results for these two cases?


### a) 
A function is created that takes as parameters a matrix $A$ and a vector $b$, which represent the constraints of a linear programming problem in standard form. The function returns a dictionary with two keys. The values of the first one, called 'Solution', correspond to a list of the basic solution vectors to the problem. The values of the key called 'Base' correspond to a list of the base corresponding to those basic solution vectors.

In [15]:
def func(A,b):
    
    # A dictionary is created whose keys and values are both empty lists.
    directory = {'Solution':[], 'Base':[]}
    
    # The dimension of the input matrix and vector is determined.
    sizeA = np.shape(A) # m equations
    sizeb = np.shape(b) # n unknowns
    
    # The combinations function is used to find all possible combinations that can be made with the m equations and n 
    # unknowns. The components of each tuple returned by the function indicate which columns of the matrix are eliminated, 
    # that is, which variables are being set to zero to solve the resulting system of equations. The number of combinations 
    # obtained is the number of basic solutions that the optimization problem has.
    combs = combinations(list(range(0,sizeA[1])),(sizeA[1]-sizeA[0]))
    
    # We are going to examine all possible combinations of variables set to zero.
    for item in list(combs):
        
        # A copy of the matrix is created, called A_1
        A_1 = A.copy()
        
        # The item-th columns (numbers of the tuple) are removed (indicated by axis=1) from A_1, that is, the variables 
        # corresponding to those columns are set to zero.
        A_1 = np.delete(A_1,item,axis=1)
        
        # Another copy of the matrix is created, called A_2
        A_2 = A.copy()
        
        # The item-th columns of A_2 are set to zero to indicate how the system looks when those variables are set to zero. 
        # These are the bases corresponding to each basic solution.
        A_2[:,item] = 0
        
        # The basic solution is given by the solution of the resulting system when the m-n variables are set to zero. If the 
        # matrix of the resulting system of equations is singular, the word 'Singular' is added to the keys, and its 
        # respective base to the values.
        if (np.linalg.det(A_1)==0):
            directory['Solution'].append('Singular Matrix')
            directory['Base'].append(A_2)
        # If the matrix can be inverted, the basic solution is found without any problem. After finding these values, the 
        # columns that were previously removed are added back, and a zero is added as the item-th component of each basic 
        # solution.
        else:
            ans = np.dot(np.linalg.inv(A_1),b)
            ans = np.round(ans,3)
            # This loop is used to traverse the item-th components of the basic solutions (the ones that were previously 
            # removed). A zero is added to each component.
            for i in item:
                ans = np.insert(ans, i, 0, axis=0)
            # The basic solutions are added to the 'Solution' key of the dictionary, and the resulting matrices to the 'Base' 
            # value.
            directory['Solution'].append(ans)
            directory['Base'].append(A_2)
                
                               
    return directory


Information on how to use the `insert` function can be found [here](https://numpy.org/doc/stable/reference/generated/numpy.insert.html).

### b)

A matrix $A$ of full rank (number of linearly independent columns equal to the number of columns) and a matrix $B$ of deficient rank with a number of linearly independent columns different from the number of columns were defined. Similarly, an arbitrary vector $b$ was created to test the previously created function with each of these constraint matrices.

The solutions to these two problems are displayed in an ordered table with the help of the *pandas* library.

In [16]:
A = np.array([[3,4,7,10,11],[0,7,12,21,8],[1,15,5,18,-9]]) # Full Rank
B = np.array([[3,4,7,10,11],[6,8,14,20,22],[1,15,5,18,-9]]) # Deficient Rank

b = np.array([[1],[4],[-7]])

sol1 = func(A,b)
sol2 = func(B,b)

out1 = pd.DataFrame(sol1)
display(out1)
 
out2 = pd.DataFrame(sol2)
display(out2)

,Solution,Base
0,"[[0.0], [0.0], [5.07], [-2.314], [-1.032]]","[[0, 0, 7, 10, 11], [0, 0, 12, 21, 8], [0, 0, ..."
1,"[[0.0], [-1.187], [0.0], [0.592], [-0.016]]","[[0, 4, 0, 10, 11], [0, 7, 0, 21, 8], [0, 15, ..."
2,"[[0.0], [-0.945], [1.033], [0.0], [-0.223]]","[[0, 4, 7, 0, 11], [0, 7, 12, 0, 8], [0, 15, 5..."
3,"[[0.0], [-1.205], [-0.079], [0.637], [0.0]]","[[0, 4, 7, 10, 0], [0, 7, 12, 21, 0], [0, 15, ..."
4,"[[-1.631], [0.0], [0.0], [-0.021], [0.555]]","[[3, 0, 0, 10, 11], [0, 0, 0, 21, 8], [1, 0, 0..."
5,"[[-1.646], [0.0], [-0.046], [0.0], [0.569]]","[[3, 0, 7, 0, 11], [0, 0, 12, 0, 8], [1, 0, 5,..."
6,"[[-1.061], [0.0], [1.772], [-0.822], [0.0]]","[[3, 0, 7, 10, 0], [0, 0, 12, 21, 0], [1, 0, 5..."
7,"[[-1.576], [-0.04], [0.0], [0.0], [0.535]]","[[3, 4, 0, 0, 11], [0, 7, 0, 0, 8], [1, 15, 0,..."
8,"[[-0.045], [-1.154], [0.0], [0.575], [0.0]]","[[3, 4, 0, 10, 0], [0, 7, 0, 21, 0], [1, 15, 0..."
9,"[[-0.463], [-0.679], [0.729], [0.0], [0.0]]","[[3, 4, 7, 0, 0], [0, 7, 12, 0, 0], [1, 15, 5,..."


,Solution,Base
0,Singular Matrix,"[[0, 0, 7, 10, 11], [0, 0, 14, 20, 22], [0, 0,..."
1,Singular Matrix,"[[0, 4, 0, 10, 11], [0, 8, 0, 20, 22], [0, 15,..."
2,Singular Matrix,"[[0, 4, 7, 0, 11], [0, 8, 14, 0, 22], [0, 15, ..."
3,Singular Matrix,"[[0, 4, 7, 10, 0], [0, 8, 14, 20, 0], [0, 15, ..."
4,Singular Matrix,"[[3, 0, 0, 10, 11], [6, 0, 0, 20, 22], [1, 0, ..."
5,Singular Matrix,"[[3, 0, 7, 0, 11], [6, 0, 14, 0, 22], [1, 0, 5..."
6,Singular Matrix,"[[3, 0, 7, 10, 0], [6, 0, 14, 20, 0], [1, 0, 5..."
7,Singular Matrix,"[[3, 4, 0, 0, 11], [6, 8, 0, 0, 22], [1, 15, 0..."
8,Singular Matrix,"[[3, 4, 0, 10, 0], [6, 8, 0, 20, 0], [1, 15, 0..."
9,Singular Matrix,"[[3, 4, 7, 0, 0], [6, 8, 14, 0, 0], [1, 15, 5,..."


The difference between these two cases is that for the full rank matrix, basic solutions that satisfy the constraints of the optimization problem can always be found.

On the other hand, for the deficient rank matrix, the resulting matrices when setting the $m-n$ variables to 0 are singular. This means that in this case, the constraints of the optimization problem are not satisfied.

# Problem 2

### Problem statement

A power plant burns coal, oil, and gas to generate electricity. Suppose that each ton of coal generates 600 kilowatt-hours, emits 20 units of sulfur dioxide, and 15 units of suspended particles, and costs $200; each ton of oil generates 550 kilowatt-hours, emits 18 units of sulfur dioxide, and 12 units of suspended particles, and costs $220; each ton of gas generates 500 kilowatt-hours, emits 15 units of sulfur dioxide, and 10 units of suspended particles, and costs $250. The Environmental Protection Agency restricts daily sulfur dioxide emissions to no more than 60 units and suspended particle emissions to no more than 75 units. If the power plant does not want to spend more than $2,000 per day on fuel, how much fuel of each type should it buy to maximize the amount of energy generated?

---


### a) Theoretical Problem Definition

#### 1) 
The variables of the problem are $x_1$, $x_2$, and $x_3$, which represent tons of coal, oil, and gas, respectively.

#### 2) 
The objective function is given by:

$$f(x_1,x_2,x_3)=600x_1+550x_2+500x_3$$

This represents the amount of energy generated daily in a plant using coal, oil, and gas as fuel.

#### 3)
The problem has the following constraints:

$$20x_1+18x_2+15x_3 \leq 60$$
$$15x_1+12x_2+10x_3 \leq 75$$
$$200x_1+220x_2+250x_3 \leq 2000$$
$$x_1,x_2,x_3 \geq 0$$

The first inequality indicates the maximum amount of sulfur dioxide units emitted by the fuels that is allowed each day. Similarly, the second inequality represents the maximum allowed units of suspended particles emissions generated by the fuels. Finally, the third inequality shows the cost constraint for purchasing the fuels, being the maximum money to spend on buying the three types of fuel.

#### 4)
The problem is defined without slack variables:

$$ \left[ \begin{array}{ccc}
20 & 18 & 15 \\
15 & 12 & 10 \\
200 & 220 & 250
\end{array} \right]
\left[ \begin{array}{ccc}
x_1\\
x_2\\
x_3
\end{array} \right] \leq
\left[ \begin{array}{ccc}
60\\
75\\
2000
\end{array} \right]
$$

The problem is defined with slack variables (in standard form):

$$ \left[ \begin{array}{ccc}
20 & 18 & 15 & 1 & 0 & 0\\
15 & 12 & 10 & 0 & 1 & 0\\
200 & 220 & 250 & 0 & 0 & 1
\end{array} \right]
\left[ \begin{array}{ccc}
x_1\\
x_2\\
x_3\\
y_1\\
y_2\\
y_3
\end{array} \right] =
\left[ \begin{array}{ccc}
60\\
75\\
2000
\end{array} \right]
$$


### b) 
A function is defined that receives the constraints of the linear program in standard form as a parameter.

This function carries out the following steps:

1. All basic solutions of the linear problem are found using the function from point 1.

2. Feasible basic solutions are filtered among the basic solutions.

3. The cost function is evaluated with each of the feasible basic solutions.

4. The `display()` function is used to show all basic solutions, feasible basic solutions, optimal, and worst solutions in an ordered manner.

To implement this function, the *inList* function was used, taken from [StackOverflow](https://stackoverflow.com/questions/23979146/check-if-numpy-array-is-in-list-of-numpy-arrays).

In [17]:
def inList(array, lista):
    for element in lista:
        if np.array_equal(element, array):
            return True
    return False

The problem and objective function are defined in code:

In [18]:
# Declaration of the constraint matrix and its respective vector of upper limits.
A = np.array([[20,18,15,1,0,0],[15,12,10,0,1,0],[200,220,250,0,0,1]])
b = np.array([[60],[75],[2000]])

# Declaration of the objective function as the negative of the problem's function, as to be in standard form,
# the function needs to be minimized.
def objective_function(x):
    return -600*x[0]-550*x[1]-500*x[2]

Definition of the function

In [19]:
def fun(A,b):
    
    # A dictionary is created with two keys. The values of the 'Feasible' key correspond to the feasible basic solutions. 
    # The values of the 'Base' key correspond to the bases corresponding to the feasible basic solutions.
    feasible = {'Feasible': [], 'Base': []}
    
    # The function created in point 1 is called to determine which are the basic solutions.
    res1 = func(A,b)
    
    # The basic solutions correspond to the values of the 'Solution' key of the dictionary delivered by the function in 
    # point 1. The bases of these basic solutions correspond to the values of the 'Base' key of the dictionary delivered
    # by the function in point 1.
    basic_solutions = res1['Solution']
    basic_bases = res1['Base']

    # The feasible basic solutions correspond to the values of the 'Solution' key of the dictionary created earlier. 
    # The bases of these feasible basic solutions correspond to the values of the 'Base' key of the dictionary created
    # earlier.
    feasible_solutions = feasible['Feasible']
    feasible_bases = feasible['Base']
   
    # Three empty lists are created. In the 'feval' list, the values of the objective function evaluated at the basic 
    # solutions are saved. In the 'feval_feasible' list, the values of the objective function evaluated at the feasible 
    # basic solutions are saved. In the 'types' list, the different classifications that can be given to the solutions 
    # of the optimization problem are saved. These strings can be 'Best', 'Worst', 'Feasible basic', and 'Basic solution'.
    feval = []
    feval_feasible = []
    types = []
    
    # All basic solutions are traversed. If all components of the vector corresponding to a basic solution are greater 
    # than or equal to zero, this solution is basic and therefore the value of this basic solution is added to the 
    # dictionary of feasible basic solutions. Similarly, the value of the base corresponding to that basic solution is 
    # added to the dictionary.
    for solution in basic_solutions:
        if all(x>=0 for x in solution) == True:
            feasible['Feasible'].append(solution)
            feasible['Base'].append(res1.get('Base', solution))
        else:
            pass

    # All basic solutions are traversed. The value of the objective function is evaluated at each of the basic solutions. 
    # This obtained value is added to the 'feval' list.
    for basic_solution in basic_solutions:
        value = objective_function(basic_solution)
        feval.append(value)
        
    # All feasible basic solutions are traversed. The value of the objective function is evaluated at each of the feasible 
    # basic solutions. This obtained value is added to the 'feval_feasible' list.
    for feasible_solution in feasible_solutions:
        feasible_value = objective_function(feasible_solution)
        feval_feasible.append(feasible_value)
        
    # Both the maximum and minimum value of the objective function evaluated at the feasible basic solutions are found.
    maximum = max(feval_feasible)
    minimum = min(feval_feasible)

    # All basic solutions are traversed
    for basic_solution in basic_solutions:
        # If the basic solution is also in the dictionary of feasible basic solutions, we know it is a feasible basic 
        # solution.
        if inList(basic_solution, feasible_solutions):
            # If by evaluating this particular feasible basic solution we obtain the maximum value found previously, this 
            # solution corresponds to the worst solution of the feasible basic solutions.
            if objective_function(basic_solution) == maximum:
                types.append("Worst")
            # If by evaluating this particular feasible basic solution we obtain the minimum value found previously, this 
            # solution corresponds to the best solution (optimal solution) of the feasible basic solutions.
            elif objective_function(basic_solution) == minimum:
                types.append("Best")
            # If by evaluating this particular feasible basic solution we obtain a value different from the maximum or 
            # minimum value found previously, this solution corresponds to only a feasible basic solution.
            else:
                types.append("Feasible basic")
        # If the basic solution is not in the dictionary of feasible basic solutions, it is only a basic solution.
        else:
            types.append("Basic solution")
            
    # Since the negative value of the objective function was being minimized, each of the values of all the basic 
    # solutions is multiplied by -1, to obtain the value that makes sense in the context of the problem, which 
    # corresponds to maximizing the objective function.
    real_values = [x * -1 for x in feval] 

    # The information obtained by the function is presented in an ordered table using the pandas library.
    table1 = pd.DataFrame({
        'Solution': basic_solutions,
        'Base': basic_bases,
        'Solution Type': types,
        'Value': real_values
    })

    display(table1)
    
    return feasible

### c) Results


#### 1) 
The amount of time it takes for the previously created function to run is counted.

In [20]:
start_feasible = time.time()
feasible_dict = fun(A,b)
end_feasible = time.time()
total_time = end_feasible - start_feasible

print(f'Time to run the fun function: {total_time} seconds')


,Solution,Base,Solution Type,Value
0,"[[0.0], [0.0], [0.0], [60.0], [75.0], [2000.0]]","[[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0...",Worst,[0.0]
1,"[[0.0], [0.0], [4.0], [0.0], [35.0], [1000.0]]","[[0, 0, 15, 0, 0, 0], [0, 0, 10, 0, 1, 0], [0,...",Best,[2000.0]
2,"[[0.0], [0.0], [7.5], [-52.5], [0.0], [125.0]]","[[0, 0, 15, 1, 0, 0], [0, 0, 10, 0, 0, 0], [0,...",Basic solution,[3750.0]
3,"[[0.0], [0.0], [8.0], [-60.0], [-5.0], [0.0]]","[[0, 0, 15, 1, 0, 0], [0, 0, 10, 0, 1, 0], [0,...",Basic solution,[4000.0]
4,"[[0.0], [3.333], [0.0], [0.0], [35.0], [1266.6...","[[0, 18, 0, 0, 0, 0], [0, 12, 0, 0, 1, 0], [0,...",Feasible basic,[1833.15]
5,"[[0.0], [6.25], [0.0], [-52.5], [0.0], [625.0]]","[[0, 18, 0, 1, 0, 0], [0, 12, 0, 0, 0, 0], [0,...",Basic solution,[3437.5]
6,"[[0.0], [9.091], [0.0], [-103.636], [-34.091],...","[[0, 18, 0, 1, 0, 0], [0, 12, 0, 0, 1, 0], [0,...",Basic solution,[5000.049999999999]
7,"[[0.0], [-6.119597140721085e+17], [7.343516568...","[[0, 18, 15, 0, 0, 0], [0, 12, 10, 0, 0, 0], [...",Basic solution,[3.0597985703605567e+19]
8,"[[0.0], [-12.5], [19.0], [0.0], [35.0], [0.0]]","[[0, 18, 15, 0, 0, 0], [0, 12, 10, 0, 1, 0], [...",Basic solution,[2625.0]
9,"[[0.0], [-1.563], [9.375], [-52.5], [0.0], [0.0]]","[[0, 18, 15, 1, 0, 0], [0, 12, 10, 0, 0, 0], [...",Basic solution,[3827.85]


Time to run the fun function: 0.020190954208374023 seconds


#### 2) 

The solution provided by the function *fun* is compared with the solution provided by the `linprog` function from the `scipy.optimize` library. The `linprog` function is tested for optimization methods *revised simplex*, *simplex*, and *interior-point*. The time it takes for each of these methods to run is recorded.


In [21]:
# Arrays corresponding to the objective function (c), the constraint matrix (A), and the upper limit vector b are declared. 
# The bounds of each of the decision variables are declared.
c = np.array([-600, -550, -500])
A = np.array([[20,18,15],[15,12,10],[200,220,25]])
b = np.array([[60],[75],[2000]])
x1_bounds = (0, None)
x2_bounds = (0, None)
x3_bounds = (0, None)

### Revised Simplex

In [22]:
start_revised = time.time()
res = linprog(c, A_ub=A, b_ub=b, bounds=np.array([x1_bounds, x2_bounds, x3_bounds]), method='revised simplex', options={"disp": True})
end_revised = time.time()
total_revised = end_revised - start_revised

print(res)
print(f'Total time with the revised-simplex method: {total_revised} seconds')

Phase Iteration Minimum Slack       Constraint Residual Objective          
1     0         60.0                0.0                 0.0                 
Phase Iteration Minimum Slack       Constraint Residual Objective          
2     0         60.0                0.0                 0.0                 
2     1         0.0                 0.0                 -1800.0             
2     2         0.0                 0.0                 -2000.0             
Optimization terminated successfully.
         Current function value: -2000.000000
         Iterations: 2
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -2000.0
       x: [ 0.000e+00  0.000e+00  4.000e+00]
     nit: 2
Total time with the revised-simplex method: 0.0024929046630859375 seconds


### Simplex

In [23]:
start_simplex = time.time()
res = linprog(c, A_ub=A, b_ub=b, bounds=np.array([x1_bounds, x2_bounds, x3_bounds]), method='simplex', options={"disp": True})
end_simplex = time.time()
total_simplex = end_simplex - start_simplex

print(res)
print(f'Total time with the simplex method: {total_simplex} seconds')


Optimization terminated successfully.
         Current function value: -2000.000000
         Iterations: 4
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -2000.0
       x: [ 0.000e+00  0.000e+00  4.000e+00]
     nit: 4
Total time with the simplex method: 0.0026378631591796875 seconds


### Interior-point

In [24]:
start_interior = time.time()
res = linprog(c, A_ub=A, b_ub=b, bounds=np.array([x1_bounds, x2_bounds, x3_bounds]), method='interior-point', options={"disp": True})
end_interior = time.time()
total_interior = end_interior - start_interior

print(res)
print(f'Total time with the interior-point method: {total_interior} seconds')

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 -1650.0             
0.1266044242263     0.1266044242263     0.1266044242263     0.8752592242131  0.1266044242266     -1626.001976117     
0.01266108624403    0.01266108624404    0.01266108624404    0.9210322525301  0.01266108624406    -1054.759228826     
0.00505630616561    0.005056306165614   0.005056306165614   0.6326796352829  0.005056306165621   -1835.584695064     
0.003569032219552   0.003569032219555   0.003569032219555   0.3205707399898  0.00356903221956    -1770.130831756     
0.0005484868775392  0.0005484868775396  0.0005484868775396  0.8485108633149  0.0005484868775404  -1872.595638374     
2.041874423905e-05  2.041874423809e-05  2.041874423809e-05  0.9855910954861  2.041874422123e-05  -1996.546816667     
1.838875572041e-09  1.838875567311e-09  1.83887556755e-09

Summarizing the recently shown information, it is clear that:

- The method that takes the least time to run is the *simplex* method.

- Following closely is the *revised simplex* method.

- The third place goes to the *interior-point* method.

- The function *fun* takes the last place.

### 3)

The solution found by all the previously implemented methods concludes that the amount of fuel of each type to be purchased to maximize the amount of generated energy is:

**0** tons of coal ($x_1$)

**0** tons of oil ($x_2$)

**4** tons of gas ($x_3$)

With these quantities of each energy source, the amount of generated energy is maximized to **2000 kWh** per day.

The reason why some of the initially found solutions become "infeasible" is because these solutions imply having some of the quantities (tons) of coal, oil, or gas in negative values, which makes no sense in the application of this optimization problem.